In [1]:
from scipy.io import loadmat
from scipy.signal import find_peaks
import numpy as np
from pathlib import Path
from typing import List, Tuple
import pickle
import matplotlib.pyplot as plt
import itertools
import torch
import torch.nn as nn
import numpy as np

Pre-process Data

In [2]:
def get_chunks(x: np.ndarray, y: np.ndarray) -> List[Tuple[np.ndarray, np.ndarray]]:
    """
    Splits the input x and y arrays on the first axis if the y value increases by more than 0.75
    """
    x = x.T[:, :3]
    y = y.T
    chunks = []
    chunk_x = []
    chunk_y = []
    for i in range(x.shape[0]):
        if i == 0:
            chunk_x.append(x[i])
            chunk_y.append(y[i])
        elif i > 0 and y[i] - y[i - 1] > 0.5 and y[i] > 0.8:
            chunks.append((np.array(chunk_x), np.array(chunk_y)))
            chunk_x = []
            chunk_y = []
        else:
            chunk_x.append(x[i])
            chunk_y.append(y[i])

    
    chunks.append((np.array(chunk_x), np.array(chunk_y)))
    return chunks

In [4]:
#grabs all .mat files from Data/New MATLAB Data
mat_files = list(Path("Data/1").glob("*.mat"))
for mat_file in mat_files:
    print(mat_file)

Data\1\01.0__Chopped_(Panasonic_7Features_SingleFile_n20_to_25degC)_V2.mat
Data\1\02.0__Chopped_(Panasonic_7Features_SingleFile_n20_to_25degC)_V2.mat
Data\1\03.0__Chopped_(Panasonic_7Features_SingleFile_n20_to_25degC)_V2.mat
Data\1\04.0__Chopped_(Panasonic_7Features_SingleFile_n20_to_25degC)_V2.mat
Data\1\05.0__Chopped_(Panasonic_7Features_SingleFile_n20_to_25degC)_V2.mat
Data\1\06.0__Chopped_(Panasonic_7Features_SingleFile_n20_to_25degC)_V2.mat
Data\1\07.0__Chopped_(Panasonic_7Features_SingleFile_n20_to_25degC)_V2.mat
Data\1\08.0__Chopped_(Panasonic_7Features_SingleFile_n20_to_25degC)_V2.mat
Data\1\09.0__Chopped_(Panasonic_7Features_SingleFile_n20_to_25degC)_V2.mat
Data\1\10.0__Chopped_(Panasonic_7Features_SingleFile_n20_to_25degC)_V2.mat


In [5]:
#grabs all .mat files from Data/New MATLAB Data
mat_files = list(Path("Data/1").glob("*.mat"))

chunks = []
for mat_file in mat_files:
    mat_data = loadmat(mat_file)
    chunks.extend(get_chunks(mat_data["X"], mat_data["Y"]))

# joins chunks thats initial value is < 0.5 and their previous chunk
for i, chunk in enumerate(chunks):
    if chunk[1][0] < 0.5 and i > 0:
        chunks[i - 1] = (np.concatenate((chunks[i - 1][0], chunk[0])), np.concatenate((chunks[i - 1][1], chunk[1])))
        del chunks[i]

# joins chunks thats final value is > 0.5 and their following chunk
for i, chunk in enumerate(chunks):
    if chunk[1][-1] > 0.5 and i < len(chunks) - 1:
        chunks[i + 1] = (np.concatenate((chunk[0], chunks[i + 1][0])), np.concatenate((chunk[1], chunks[i + 1][1])))
        del chunks[i]

In [11]:
#prints the number of chunks
print(len(chunks))
all_chunks = chunks

30


Define NN

In [12]:
class LSTMModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden_size = 16
        self.num_layers = 1
        self.lstm = nn.LSTM(3, 16, 1, batch_first=True)
        self.ln = nn.LayerNorm(16)
        self.fc2 = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(16, 1),
        )
        
    def forward(self, x):
        # out = self.fc1(x)
        out = x
        out, _ = self.lstm(out)
        out = self.ln(out)
        out = self.fc2(out)
        out = torch.clip(out, 0, 1)
        return out

In [13]:
model = LSTMModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [20]:
for chunk in all_chunks:
    for (x,y) in chunk:
        print(x)


ValueError: too many values to unpack (expected 2)

In [15]:
model.train()
for i in range(500):
    loss = 0
    for chunk in all_chunks:
        for (inputs, targets) in chunk:
            inputs = torch.from_numpy((inputs - means) / stds).float().unsqueeze(0).cuda()
            targets = torch.from_numpy(targets).float().unsqueeze(0).cuda()
            
            # Forward pass
            outputs = model(inputs)
            loss += criterion(outputs, targets) ** 0.5
    loss /= 29.

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(loss.item())

ValueError: too many values to unpack (expected 2)